In [1]:
from torchvision.models.mobilenet import mobilenet_v2
import torch.nn as nn
import cv2
from torchvision import transforms
import torch
import torchvision

CATEGORIES = ["Konrad", "Falco"]
RESIZE_M = 150
RESIZE_N = 100



def get_data(file):
    img_array = cv2.imread(file, cv2.IMREAD_COLOR)
    img_array_resized = cv2.resize(img_array,(RESIZE_M,RESIZE_N))
    return img_array_resized.reshape(-1, RESIZE_M, RESIZE_N, 1)

model = torch.load('/nfshome/lentzsch/Documents/Bachelorarbeit/Bachelorthesis/class_sample/model.pt')



In [2]:
data = get_data('/nfshome/lentzsch/Documents/Bachelorarbeit/Test/falco031.png')
data_tensor = torch.from_numpy(data)
data_tensor = data_tensor.permute(3, 0, 1, 2)




In [4]:
model.eval()
model.
output = model(data_tensor.float().cuda())
print(output)

tensor([[-1.8490e+00,  3.7672e-01, -4.4966e+00, -5.7096e+00,  2.6009e-01,
          7.3482e-01, -7.2077e-01, -2.7002e+00, -3.6026e-01,  2.0368e-01,
         -7.4080e+00,  3.4215e-01, -3.3591e+00, -6.3379e+00, -6.7866e+00,
         -4.2057e+00, -2.1936e+00, -6.3646e+00, -5.1778e+00, -2.3770e+00,
         -4.4376e+00, -3.3152e-02, -3.0710e+00,  3.5424e+00, -6.9333e-01,
         -3.0513e+00, -8.8948e+00, -1.2669e+01, -1.1705e+01, -5.0558e+00,
         -7.3665e-01,  3.1393e+00,  3.2536e-01,  1.5857e+00,  1.5092e+00,
          1.3386e+00,  9.6019e-01,  9.2260e-01, -1.1016e+00, -5.4428e+00,
          1.3516e+00, -5.8300e+00, -8.4775e+00, -8.3048e+00, -7.5917e+00,
         -3.8002e+00, -1.5716e+00, -1.3097e+00,  3.0192e+00, -3.6389e+00,
         -6.9422e+00, -2.5775e+00, -4.0905e+00, -5.5978e+00, -5.0974e+00,
         -1.7905e+00, -6.6459e+00, -1.3174e+01, -7.0885e+00, -5.6001e-02,
          1.4901e-02, -4.0416e+00, -5.5066e+00, -4.2650e+00, -5.5112e+00,
         -4.9089e+00, -5.1579e+00, -8.